### Impact Analysis of Series 'Euphoria'
#### Analysis based on posts and comments on the `r/euphoria` subreddit

Analysis plan:
1. topic modeling based on comments of drug-related posts on r/euphoria
2. network analysis between users of drug and euphoria communities
3. aspect based sentiment analysis of euphoria drug-related comments
4. topic modeling based on comments that refer to euphoria on r/opioids, r/cannabis, r/benzodiazepenes

In [ ]:
# analysis plan:
# 1. sentiment analysis of reviews for each film with VADER
# 2. aspect analysis of reviews for each film
# 3. sentiment analysis of reviews with dense embeddings - doc2vec
# 4. emotion analysis with NRC

In [ ]:
# lexical based sentiment analysis
# AFINN and VADER
import pandas as pd
from afinn import Afinn
from nltk.sentiment.vader import SentimentIntensityAnalyzer